https://mlflow.org/docs/latest/ml/getting-started/hyperparameter-tuning


# 1단계: 데이터 준비


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow import keras
import mlflow
from mlflow.models import infer_signature
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

# Load the wine quality dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)
print("데이터 다운로드 완료")
print(data[:5])
print(len(data))

# Create train/validation/test splits
train, test = train_test_split(data, test_size=0.25, random_state=42)
train_x = train.drop(["quality"], axis=1).values
train_y = train[["quality"]].values.ravel()
test_x = test.drop(["quality"], axis=1).values
test_y = test[["quality"]].values.ravel()

# Further split training data for validation
train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=42
)

print("데이터 분할 완료")

# Create model signature for deployment
signature = infer_signature(train_x, train_y)

print("모델 시그니처 생성 완료")

# 2단계: 모델 아키텍처 정의


In [ ]:
def create_and_train_model(learning_rate, momentum, epochs=10):
    """
    Create and train a neural network with specified hyperparameters.

    Returns:
        dict: Training results including model and metrics
    """
    # Normalize input features for better training stability
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)

    # Define model architecture
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dropout(0.2),  # Add regularization
            keras.layers.Dense(32, activation="relu"),
            keras.layers.Dense(1),
        ]
    )

    # Compile with specified hyperparameters
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=learning_rate, momentum=momentum
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    # Train with early stopping for efficiency
    early_stopping = keras.callbacks.EarlyStopping(
        patience=3, restore_best_weights=True
    )

    # Train the model
    history = model.fit(
        train_x,
        train_y,
        validation_data=(valid_x, valid_y),
        epochs=epochs,
        batch_size=64,
        callbacks=[early_stopping],
        verbose=0,  # Reduce output for cleaner logs
    )

    # Evaluate on validation set
    val_loss, val_rmse = model.evaluate(valid_x, valid_y, verbose=0)

    return {
        "model": model,
        "val_rmse": val_rmse,
        "val_loss": val_loss,
        "history": history,
        "epochs_trained": len(history.history["loss"]),
    }

# 3단계: 하이퍼파라미터 최적화 설정


In [ ]:
def objective(params):
    """
    Objective function for hyperparameter optimization.
    This function will be called by Hyperopt for each trial.
    """
    with mlflow.start_run(nested=True):
        # Log hyperparameters being tested
        mlflow.log_params(
            {
                "learning_rate": params["learning_rate"],
                "momentum": params["momentum"],
                "optimizer": "SGD",
                "architecture": "64-32-1",
            }
        )

        # Train model with current hyperparameters
        result = create_and_train_model(
            learning_rate=params["learning_rate"],
            momentum=params["momentum"],
            epochs=15,
        )

        # Log training results
        mlflow.log_metrics(
            {
                "val_rmse": result["val_rmse"],
                "val_loss": result["val_loss"],
                "epochs_trained": result["epochs_trained"],
            }
        )

        # Log the trained model
        mlflow.tensorflow.log_model(
            result["model"], name="model", signature=signature
        )

        # Log training curves as artifacts
        import matplotlib.pyplot as plt

        plt.figure(figsize=(12, 4))

        plt.subplot(1, 2, 1)
        plt.plot(
            result["history"].history["loss"], label="Training Loss"
        )
        plt.plot(
            result["history"].history["val_loss"],
            label="Validation Loss",
        )
        plt.title("Model Loss")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(
            result["history"].history["root_mean_squared_error"],
            label="Training RMSE",
        )
        plt.plot(
            result["history"].history["val_root_mean_squared_error"],
            label="Validation RMSE",
        )
        plt.title("Model RMSE")
        plt.xlabel("Epoch")
        plt.ylabel("RMSE")
        plt.legend()

        plt.tight_layout()
        plt.savefig("training_curves.png")
        mlflow.log_artifact("training_curves.png")
        plt.close()

        # Return loss for Hyperopt (it minimizes)
        return {"loss": result["val_rmse"], "status": STATUS_OK}


# Define search space for hyperparameters
search_space = {
    "learning_rate": hp.loguniform(
        "learning_rate", np.log(1e-5), np.log(1e-1)
    ),
    "momentum": hp.uniform("momentum", 0.0, 0.9),
}

print("Search space defined:")
print("- Learning rate: 1e-5 to 1e-1 (log-uniform)")
print("- Momentum: 0.0 to 0.9 (uniform)")

# 4단계: 하이퍼파라미터 최적화 실행


In [ ]:
# Create or set experiment
experiment_name = "wine-quality-optimization"
mlflow.set_experiment(experiment_name)

print(
    f"Starting hyperparameter optimization experiment: {experiment_name}"
)
print("This will run 15 trials to find optimal hyperparameters...")

with mlflow.start_run(run_name="hyperparameter-sweep"):
    # Log experiment metadata
    mlflow.log_params(
        {
            "optimization_method": "Tree-structured Parzen Estimator (TPE)",
            "max_evaluations": 15,
            "objective_metric": "validation_rmse",
            "dataset": "wine-quality",
            "model_type": "neural_network",
        }
    )

    # Run optimization
    trials = Trials()
    best_params = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=15,
        trials=trials,
        verbose=True,
    )

    # Find and log best results
    best_trial = min(trials.results, key=lambda x: x["loss"])
    best_rmse = best_trial["loss"]

    # Log optimization results
    mlflow.log_params(
        {
            "best_learning_rate": best_params["learning_rate"],
            "best_momentum": best_params["momentum"],
        }
    )
    mlflow.log_metrics(
        {
            "best_val_rmse": best_rmse,
            "total_trials": len(trials.trials),
            "optimization_completed": 1,
        }
    )

# 5단계: MLflow UI에서 결과 분석

## 테이블 뷰

1. 실험으로 이동 : "wine-quality-optimization"을 클릭하세요.
2. 주요 열 추가 : "열"을 클릭하고 다음을 추가합니다.

- Metrics | val_rmse
- Parameters | learning_rate
- Parameters | momentum

3. 성과별 정렬 : val_rmse열 머리글을 클릭하여 가장 우수한 성과별로 정렬합니다.

## 시각적

1. 차트 보기로 전환 : "차트" 탭을 클릭하세요
2. 평행 좌표 플롯 생성 :

- "평행 좌표"를 선택하세요
- 좌표 learning_rate로 추가momentum
- val_rmse메트릭으로 설정

3. 시각화를 해석하세요 :

- 파란색 선 = 더 나은 성능의 실행
- 빨간색 선 = 성능이 저하된 실행
- 성공적인 매개변수 조합에서 패턴을 찾으세요

### 주요 통찰력

- 학습률 패턴 : 너무 높으면 불안정해지고 너무 낮으면 수렴이 느립니다.
- 모멘텀 효과 : 중간 모멘텀(0.3-0.7)이 가장 효과적입니다.
- 학습 곡선 : 모델이 제대로 수렴되었는지 확인하기 위해 아티팩트를 확인합니다.


# 6단계: 가장 좋은 모델 등록

최고의 모델을 프로덕션에 홍보할 시간입니다.

1. 가장 좋은 실행 찾기 : 표 보기에서 가장 낮은 실행을 클릭합니다.val_rmse

2. 모델 아티팩트로 이동 : "아티팩트" 섹션으로 스크롤합니다.

3. 모델 등록 :

- 모델 폴더 옆에 있는 "모델 등록"을 클릭하세요.
- 모델 이름을 입력하세요:wine-quality-predictor
- 설명 추가: "와인 품질 예측을 위한 최적화된 신경망"
- "등록"을 클릭하세요

4. 모델 수명 주기 관리 :

- MLflow UI의 "모델" 탭으로 이동
- 등록된 모델을 클릭하세요
- 테스트를 위한 "스테이징" 단계로 전환
- 필요에 따라 태그와 설명을 추가하세요


# 7단계: 로컬로 모델 배포

```bash
# Serve the model (choose the version number you registered)
$ mlflow models serve -m "models:/wine-quality-predictor/1" --port 5002
```

## 테스트

```bash
# Test with a sample wine
curl -X POST http://localhost:5002/invocations \
  -H "Content-Type: application/json" \
  -d '{
    "dataframe_split": {
      "columns": [
        "fixed acidity", "volatile acidity", "citric acid", "residual sugar",
        "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density",
        "pH", "sulphates", "alcohol"
      ],
      "data": [[7.0, 0.27, 0.36, 20.7, 0.045, 45, 170, 1.001, 3.0, 0.45, 8.8]]
    }
  }'
```

## 예상 응답

```json
{
  "predictions": [5.31]
}
```


In [ ]:
# 파이썬으로 테스트하기
import requests
import json


# Prepare test data
test_wine = {
    "dataframe_split": {
        "columns": [
            "fixed acidity",
            "volatile acidity",
            "citric acid",
            "residual sugar",
            "chlorides",
            "free sulfur dioxide",
            "total sulfur dioxide",
            "density",
            "pH",
            "sulphates",
            "alcohol",
        ],
        "data": [
            [
                7.0,
                0.27,
                0.36,
                20.7,
                0.045,
                45,
                170,
                1.001,
                3.0,
                0.45,
                8.8,
            ]
        ],
    }
}

# Make prediction request
response = requests.post(
    "http://localhost:5002/invocations",
    headers={"Content-Type": "application/json"},
    data=json.dumps(test_wine),
)

prediction = response.json()
print(f"Predicted wine quality: {prediction['predictions'][0]:.2f}")

# 8단계: 프로덕션

클라우드 배포를 위한 Docker 컨테이너를 만듭니다.

```bash
# Build Docker image
mlflow models build-docker \
  --model-uri "models:/wine-quality-predictor/1" \
  --name "wine-quality-api"
```

## 테스트

```bash
# Run the container
docker run -p 5003:8080 wine-quality-api

# Test in another terminal
curl -X POST http://localhost:5003/invocations \
  -H "Content-Type: application/json" \
  -d '{
    "dataframe_split": {
      "columns": ["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol"],
      "data": [[7.0, 0.27, 0.36, 20.7, 0.045, 45, 170, 1.001, 3.0, 0.45, 8.8]]
    }
  }'
```

## ECR wjwkd

```bash
# Example: Push to ECR and deploy to ECS
aws ecr create-repository --repository-name wine-quality-api
docker tag wine-quality-api:latest <your-account>.dkr.ecr.us-east-1.amazonaws.com/wine-quality-api:latest
docker push <your-account>.dkr.ecr.us-east-1.amazonaws.com/wine-quality-api:latest
```
